In [183]:
import pandas as pd
from sklearn.impute import KNNImputer

In [184]:
data = pd.read_csv('../data/data.csv', sep=',', encoding='utf8', parse_dates=['bdate_cl'])
data

C:\Users\38093\AppData\Local\Temp\ipykernel_90044\779075293.py:1: DtypeWarning: Columns (9,10,11,13,26,27,28,29,30,31,33,34,35,36,37,38,41,43,51,54,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/data.csv', sep=',', encoding='utf8', parse_dates=['bdate_cl'])


,#,orc_id,fio,age,bdate,bplace,nationality,gender,mstatus,address,...,bucha,description,mstatus_cl,edu_lvl_cl,mil_rank_cl,bdate_cl,mil_date_cl,pass_date_cl,contract_date_cl,contract_end_date_cl
0,1,439,Абрамцов Владимир Сергеевич,30.0,1991-08-19,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,Сержант,1991-08-19,NaN,2015-08-04,NaN,NaN
1,2,452,Фаттахов Вагиф Нариман-оглы,32.0,1990-04-01,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,Сержант,1990-04-01,NaN,2010-05-13,NaN,NaN
2,3,456,Тушевский Иван Георгиевич,25.0,1996-02-09,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,Сержант,1996-02-09,NaN,2010-03-11,NaN,NaN
3,4,458,Мезенин Виктор Вячеславович,27.0,1994-12-15,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,Сержант,1994-12-15,NaN,2009-03-05,NaN,NaN
4,5,97567,Худяк Денис Сергеевич,33.0,1989-02-06,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,NaN,1989-02-06,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110067,110068,99495,Черненко Константин Игоревич,33.0,1988-08-28,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,NaN,1988-08-28,NaN,NaN,NaN,NaN
110068,110069,109315,Арчинеков Вячеслав Владимирович,NaN,1982-09-23,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,Матрос,1982-09-23,NaN,2015-06-29,NaN,NaN
110069,110070,109320,Мирошниченко Александр Анатольевич,NaN,1985-02-28,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,NaN,1985-02-28,NaN,NaN,NaN,NaN
110070,110071,109686,Сорокин Дмитрий Андреевич,NaN,1991-08-06,NaN,NaN,m,NaN,NaN,...,False,NaN,NaN,NaN,NaN,1991-08-06,NaN,NaN,NaN,NaN


In [185]:
data['nationality'] = data['nationality'].astype(str)

In [186]:
def get_missing_data_stats(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isna().mean() * 100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [187]:
data = data[['age', 'nationality', 'gender', 'mstatus_cl', 'mil_rank_cl', 'edu_lvl_cl']]
data

,age,nationality,gender,mstatus_cl,mil_rank_cl,edu_lvl_cl
0,30.0,nan,m,NaN,Сержант,NaN
1,32.0,nan,m,NaN,Сержант,NaN
2,25.0,nan,m,NaN,Сержант,NaN
3,27.0,nan,m,NaN,Сержант,NaN
4,33.0,nan,m,NaN,NaN,NaN
...,...,...,...,...,...,...
110067,33.0,nan,m,NaN,NaN,NaN
110068,NaN,nan,m,NaN,Матрос,NaN
110069,NaN,nan,m,NaN,NaN,NaN
110070,NaN,nan,m,NaN,NaN,NaN


In [188]:
def clear_mil_rank(string):
    match string:
        case "Капитан 3 ранга":
            return 1
        case "Старший Прапорщик":
            return 2
        case "Полковник":
            return 3
        case "Лейтенант":
            return 4
        case "Старший Сержант":
            return 5
        case "Капитан-лейтенант":
            return 6
        case "Старшина":
            return 7
        case "Подполковник":
            return 8
        case "Капитан":
            return 9
        case "Генерал-майор":
            return 10
        case "Старший Лейтенант":
            return 11
        case "Сержант":
            return 12
        case "Капитан 1 ранга":
            return 13
        case "Мичман":
            return 14
        case "Младший Сержант":
            return 15
        case "Ефрейтор":
            return 16
        case "Прапорщик":
            return 17
        case "Страший Матрос":
            return 18
        case "Матрос":
            return 19
        case "Капитан 2 ранга":
            return 20
        case "Майор":
            return 21
        case "Старший Матрос":
            return 22
        case "Старший Мичман":
            return 23
        case "Старшина Сержант":
            return 24
        case "Рядовой":
            return 25
        case "Контр-адмирал":
            return 26
        case "Старший Офицер":
            return 27

    return 0

def clear_edu_lvl(string):
    match string:
        case "Среднее техническое":
            return 1
        case "Среднее специальное":
            return 2
        case "Среднее профессиональное":
            return 3
        case "Среднее полное":
            return 4
        case "Среднее":
            return 5
        case "Начальное профессиональное":
            return 6
        case "Высшее":
            return 7
        case "ВУЦ":
            return 8
        case "ВВУЗ":
            return 9

    return 0

def clear_nationality(string):
    if (string == "русский" or string == "руский" or string == "Русский" or string == "русская"):
        return "русский"
    elif (string == "бурят " or string == "Бурят " or string == "бурят"):
        return "бурят"
    elif (string == "азербайджанец" or string == "азербейджанец"):
        return "азербайджанец"
    else:
        return string.lower()

data['edu_lvl'] = data['edu_lvl_cl'].apply(clear_edu_lvl)
data['mil_rank'] = data['mil_rank_cl'].apply(clear_mil_rank)
data['nationality'] = data['nationality'].apply(clear_nationality)
data = data.drop(columns=['edu_lvl_cl', 'mil_rank_cl'])
data

C:\Users\38093\AppData\Local\Temp\ipykernel_90044\197783487.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['edu_lvl'] = data['edu_lvl_cl'].apply(clear_edu_lvl)
C:\Users\38093\AppData\Local\Temp\ipykernel_90044\197783487.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mil_rank'] = data['mil_rank_cl'].apply(clear_mil_rank)
C:\Users\38093\AppData\Local\Temp\ipykernel_90044\197783487.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,age,nationality,gender,mstatus_cl,edu_lvl,mil_rank
0,30.0,nan,m,NaN,0,12
1,32.0,nan,m,NaN,0,12
2,25.0,nan,m,NaN,0,12
3,27.0,nan,m,NaN,0,12
4,33.0,nan,m,NaN,0,0
...,...,...,...,...,...,...
110067,33.0,nan,m,NaN,0,0
110068,NaN,nan,m,NaN,0,19
110069,NaN,nan,m,NaN,0,0
110070,NaN,nan,m,NaN,0,0


In [189]:
data = data.reset_index(drop=True)
data = pd.get_dummies(data)
columns = data.columns
data

In [191]:
imputer = KNNImputer(n_neighbors=20)
data = imputer.fit_transform(data)
data

array([[30. ,  0. , 12. , ...,  0. ,  0. ,  0. ],
       [32. ,  0. , 12. , ...,  0. ,  0. ,  0. ],
       [25. ,  0. , 12. , ...,  0. ,  0. ,  0. ],
       ...,
       [37.3,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [37.3,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [31.6,  0. , 16. , ...,  0. ,  0. ,  0. ]])

In [194]:
df = pd.DataFrame(data, columns=columns)
df

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,30.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,32.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110067,33.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
110068,26.4,0.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
110069,37.3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
110070,37.3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
df.to_csv('../data/preprocessed.csv')

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'